In [1]:
import requests
import logging
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
r = requests.get('https://daddyleagues.com/dbml/player/45732844')

c = r.content

soup = BeautifulSoup(c, 'html.parser')

general_info = soup.find_all('div', {'class': 'col-10 py-3 rounded-right cfm-player-info'})

In [3]:
general_info[0].find_all('div', {'class': 'col-3'})

[<div class="col-3">
 <h3 class="m-0">5'11''</h3>
 <p class="m-0 text-muted">Height</p>
 </div>,
 <div class="col-3">
 <h3 class="m-0">  215 </h3>
 <p class="m-0 text-muted">Weight</p>
 </div>,
 <div class="col-3">
 <h3 class="m-0"> 35 </h3>
 <p class="m-0 text-muted">Age</p>
 </div>,
 <div class="col-3">
 <h3 class="m-0">12 yrs </h3>
 <p class="m-0 text-muted">Exp</p>
 </div>]

In [9]:
def cfm_web_scraper(link):
    r = requests.get(link)

    c = r.content

    soup = BeautifulSoup(c, 'html.parser')

    player_info = soup.find_all('div', {'class': 'col-10 py-3 rounded-right cfm-player-info'})
    
    d = {}
    
    name_info_list = player_info[0].find('div', {'class': 'h2 mt-0'}).text.split(' ')
    general_info_list = player_info[0].find_all('div', {'class': 'col-3'})
    dev_info_list = player_info[0].find_all('div', {'class': 'col-4'})
    
    if len(name_info_list) == 11:
        d['Name'] = name_info_list[1] + ' ' + name_info_list[2]
        d['Position'] = name_info_list[4]
        d['Overall'] = int(name_info_list[8])
    else:
        if name_info_list[1] == 'Ha':
            name_info_list.pop(2)
            name_info_list[1] = 'Ha\'Sean'
        d['Name'] = name_info_list[1] + ' ' + name_info_list[2] + ' ' + name_info_list[3]
        d['Position'] = name_info_list[5]
        if name_info_list[1] == "Ha'Sean":
            d['Overall'] = name_info_list[8]
        else:
            d['Overall'] = name_info_list[9]

    d['Height'] = general_info_list[0].text.replace('\n', '')[0:5]
    d['Weight'] = int(general_info_list[1].text.replace('\n', '')[0:3])
    d['Age'] = int(general_info_list[2].text.replace('\n', '')[0:2])
    d['Years in League'] = general_info_list[3].text.replace('\n', '')[0:6]

    d['Dev'] = dev_info_list[1].text.replace('\n', '')
    d['Trade Value'] = int(re.sub("[^0-9]", "", dev_info_list[2].text.replace('\n', '')))
    
    return d

In [10]:
d = cfm_web_scraper('https://daddyleagues.com/dbml/player/45731878')
d

{'Name': "Ha'Sean Clinton-Dix ",
 'Position': '#36',
 'Overall': '80',
 'Height': "6'1''",
 'Weight': 2,
 'Age': 3,
 'Years in League': '10 yrs',
 'Dev': 'Star  Dev',
 'Trade Value': 23}

In [11]:
pd.DataFrame(d, index=[0])

,Name,Position,Overall,Height,Weight,Age,Years in League,Dev,Trade Value
0,Ha'Sean Clinton-Dix,#36,80,6'1'',2,3,10 yrs,Star Dev,23
